# Model 1 - custom MCMC

This example builds upon the previous notebook, so we'll be more terse about some of the stuff already covered, and focus on how to do posterior estimation using MCMC.

![Model1](img/model1.png)
Taken from Figure 6 of Vincent (2015).

## Define the posterior density function

In [1]:
using StatsFuns

# define Φ() as the cumulative normal distribution function
Φ(x) = normcdf.(0, 1, x)

# define the deterministic node PC, corresponds to Equation 2 in Vincent (2015)
PC(Δμ, σ²::Number) = Φ(Δμ/sqrt(2σ²));

log_prior(σ²) = log(1/1000)

function log_likelihood(σ²::Number, Δμ::Array{Float64,1}, k::Array{Int64,1}, T::Integer)
    log_likelihood = sum(binomlogpdf.(T, PC(Δμ, σ²), k))
end

log_posterior(σ², Δμ, k, T::Integer) = log_prior(σ²) + log_likelihood(σ², Δμ, k, T)

posterior_density(σ², Δμ, k, T) = exp(log_posterior(σ², Δμ, k, T::Integer))

posterior_density (generic function with 1 method)

## Define our data

In [2]:
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000];

## Define the MH Algorithm

In [3]:
using Distributions

function mhAlgorithm(pdf, Δμ::Array{Float64,1}, k::Array{Int64,1}, T::Number, θ₀::Number, nSamples=10^4, proposalσ=0.1)
    
    accepted = 0
    samples = zeros(Float64, nSamples,1)
    samples[1] = θ₀

    for n ∈ 2:nSamples
        old_sample = samples[n-1]
        old_posterior = pdf(old_sample, Δμ, k, T)

        # suggest a new sample, but θ can't be less than or equal to zero
        new_sample = -Inf
        while new_sample ≤ 0
            new_sample = rand(Normal(old_sample,proposalσ))
        end
        new_posterior = pdf(new_sample, Δμ, k, T)

        # maybe accept new sample
        if new_posterior > old_posterior
            samples[n] = new_sample
            accepted += 1
        else
            u = rand(Normal(0,1))
            if u < new_posterior/old_posterior
                samples[n] = new_sample
                accepted += 1
            else
                samples[n] = old_sample
            end
        end

#         if rem(n,5000)==0
#             println("$n of $nSamples")
#         end
    end

    acceptance_rate = accepted / nSamples;
    println("Acceptance rate: $(acceptance_rate*100) %")
    return samples
end

mhAlgorithm (generic function with 3 methods)

# Generate samples using the MH algorithm

In [4]:
@time samples = mhAlgorithm(posterior_density, data["Δμ"], data["k"], data["T"], 2);

Acceptance rate: 85.46000000000001 %
  0.426200 seconds (239.92 k allocations: 19.007 MiB, 3.33% gc time)


In [5]:
using Plots
plot(samples, legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2000 
 
 
 4000 
 
 
 6000 
 
 
 8000 
 
 
 10000 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
<polyline clip-path="url(#clip8502)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,155.31 32.2769,154.148 32.3317,147.936 32.3865,147.936 32.4414,139.849 32.4962,124.875 32.551,134.12 32.6058,134.12 32.6606,140.794 32.7154,139.396 
 32.7703,148.051 32.8251,144.534 32.8799,154.591 32.9347,159.836 32.9895,170.756 33.0443,171.151 33.0991,174.389 33.154,171.335 33.2088,175.052 33.2636,175.052 
 33.3184,181.799 33.3732,205.474 33.428,219.468 33.4828,231.317 33.5377,247.062 33.5925,250.164 33.6473,239.806 33.7021,233.417 33.7569,233.417 33.8117,241.588 
 33.8665,234.996 33.9214,229.839 33.9762,249.485 34.031,239.645 34.0858,250.271 34.1406,258.142 34.1954,251.635 34.2503,236.014 34.3051,244.941 34.3599,261.27 
 34.4147,263.725 34.4695,258.968 34.5243,274.709 34.5791,264.194 34.634,267.252 34.6888,268.435 34.7436,275.42 34.7984,282.103 34.8532,281.692 34.908,284.425 
 34.9628,250.737 35.0177,247.661 35.0725,253.78 35.1273,240.196 35.1821,245.94 35.2369,245.94 35.2917,237.92 35.3465,251.854 35.4014,243.251 35.4562,231.077 
 35.511,245.807 35.5658,244.344 35.6206,251.749 35.6754,256.021 35.7303,260.321 35.7851,252.018 35.8399,252.991 35.8947,264.368 35.9495,256.639 36.0043,243.302 
 36.0591,242.675 36.114,241.078 36.1688,223.915 36.2236,222.82 36.2784,241.609 36.3332,252.02 36.388,252.02 36.4428,240.832 36.4977,225.407 36.5525,207.243 
 36.6073,215.697 36.6621,234.639 36.7169,241.077 36.7717,240.477 36.8265,260.065 36.8814,254.356 36.9362,242.226 36.991,232.72 37.0458,226.142 37.1006,235.952 
 37.1554,230.737 37.2102,211.211 37.2651,222.712 37.3199,223.42 37.3747,226.876 37.4295,211.829 37.4843,203.196 37.5391,194.268 37.594,209.928 37.6488,209.928 
 37.7036,202.714 37.7584,198.506 37.8132,181.174 37.868,191.701 37.9228,190.533 37.9777,191.534 38.0325,180.338 38.0873,172.362 38.1421,163.157 38.1969,172.44 
 38.2517,174.131 38.3065,174.131 38.3614,174.131 38.4162,187.446 38.471,189.277 38.5258,189.277 38.5806,189.277 38.6354,198.227 38.6902,208.791 38.7451,219.092 
 38.7999,227.067 38.8547,227.067 38.9095,222.726 38.9643,227.327 39.0191,226.033 39.074,221.772 39.1288,223.163 39.1836,232.293 39.2384,218.652 39.2932,219.007 
 39.348,212.034 39.4028,212.034 39.4577,212.034 39.5125,226.177 39.5673,212.65 39.6221,218.641 39.6769,201.056 39.7317,205.263 39.7865,223.018 39.8414,223.018 
 39.8962,223.018 39.951,216.242 40.0058,216.242 40.0606,220.561 40.1154,220.561 40.1702,220.561 40.2251,220.561 40.2799,238.018 40.3347,229.135 40.3895,239.404 
 40.4443,247.029 40.4991,243.57 40.554,217.849 40.6088,217.849 40.6636,222.941 40.7184,209.901 40.7732,209.901 40.828,207.087 40.8828,220.886 40.9377,214.418 
 40.9925,226.25 41.0473,232.361 41.1021,231.799 41.1569,221.545 41.2117,221.545 41.2665,244.69 41.3214,252.31 41.3762,238.188 41.431,257.038 41.4858,260.015 
 41.5406,258.702 41.5954,263.486 41.6502,263.486 41.7051,250.513 41.7599,254.69 41.8147,256.207 41.8695,247.564 41.9243,247.564 41.9791,246.421 42.0339,254.483 
 42.0888,253.292 42.1436,254.154 42.1984,260.873 42.2532,267.552 42.308,269.707 42.3628,281.831 42.4177,286.155 42.4725,287.807 42.5273,296.607 42.5821,296.607 
 42.6369,300.738 42.6917,286.42 42.7465,305.127 42.8014,311.541 42.8562,307.641 42.911,307.641 42.9658,316.75 43.0206,308.292 43.0754,294.331 43.1302,280.217 
 43.1851,278.416 43.2399,289.079 43.2947,280.377 43.3495,284.294 43.4043,284.294 43.4591,288 43.5139,288 43.5688,286.592 43.6236,264.93 43.6784,248.959 
 43.7332,235.154 43.788,235.154 43.8428,223.24 43.8977,217.517 43.9525,217.517 44.0073,231.318 44.0621,212.497 44.1169,193.842 44.1717,180.72 44.2265,181.202 
 44.2814,175.077 44.3362,167.132 44.391,163.085 44.4458,163.708 44.5006,157.916 44.5554,175.258 44.6102,192.773 44.6651,183.518 44.7199,183

In [6]:
# TODO - plot histogram of samples

In [7]:
# TODO - calculate summary stats of samples

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001